## BUSINESS PROBLEM

UK-based retail company wants to segment its customers and determine a roadmap for sales and marketing activities. It thinks that marketing activities specific to customer segments that exhibit common behaviors will increase revenue.
RFM analysis will be used for segmentation. In addition, in order for the company to make a medium and long term plan, the existing customers
estimate the potential value they will provide to the company in the future.

### DATASET BACKGROUND

The dataset Online Retail II contains online sales transactions of a UK-based retail company between 01/12/2009 and 09/12/2011. The company's product catalog includes souvenirs and most of its customers are wholesalers.

8 Variable, 541.909 Observation

* InvoiceNo : Invoice Number (If this code starts with C, it means that the transaction was canceled)
* StockCode : Product Code (Unique for each product)
* Description : Product Name
* Quantity : Number of Products (How many of the products in the invoices were sold)
* InvoiceDate : Invoice Date
* UnitPrice : Invoice Price (£)
* CustomerID : Unique Customer Number
* Country : Country Name

### IMPORTINGS

In [ ]:
!pip install lifetimes

In [ ]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_ = pd.read_excel("/kaggle/input/online-retail-ii-dataset/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

### FUNCTIONS

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

### DATA UNDERSTANDING & PREPARATION

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df["StockCode"].nunique()

In [ ]:
df.groupby("Description").agg({"Quantity" : "sum"})

In [ ]:
df.groupby("Description").agg({"Invoice" : "count"}).sort_values("Invoice", ascending=False).head(5)

In [ ]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [ ]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.head()

### CALCULATING RFM METRICS

In [ ]:
today_date = df["InvoiceDate"].max() + dt.timedelta(days=2)

In [ ]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate" : lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     "Invoice" : lambda Invoice: Invoice.nunique(),
                                     "TotalPrice" : lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ["recency", "frequency", "monetary"]
rfm.reset_index(inplace=True)
rfm.head()

### CALCULATING RFM SCORES

In [ ]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, [5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, [1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, [1, 2, 3, 4, 5])

rfm["RF_Score"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

rfm.head()

### Creating & Analysing RFM Segments

In [ ]:
seg_map = {r'[1-2][1-2]': 'hibernating',
           r'[1-2][3-4]': 'at_Risk',
           r'[1-2]5': 'cant_loose',
           r'3[1-2]': 'about_to_sleep',
           r'33': 'need_attention',
           r'[3-4][4-5]': 'loyal_customers',
           r'41': 'promising',
           r'51': 'new_customers',
           r'[4-5][2-3]': 'potential_loyalists',
           r'5[4-5]': 'champions'}

rfm["Segment"] = rfm["RF_Score"].replace(seg_map, regex=True)

rfm.head()

In [ ]:
rfm_selected = rfm[rfm["Segment"].isin(["hibernating", "champions", "at_Risk"])]

rfm_selected.groupby("Segment").agg({"recency" : ["max", "min", "count", "mean"],
                                     "frequency" : ["max", "min", "count", "mean"],
                                     "monetary" : ["max", "min", "count", "mean"]})

rfm_loyal = rfm[rfm["Segment"].isin(["loyal_customers"])]["Customer ID"].astype("int").reset_index(drop=True)
rfm_loyal.head()

### CLTV PREDICTION

In [ ]:
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [ ]:
today_date = df["InvoiceDate"].max() + dt.timedelta(days=2)

### PREPARATION of DATA STRUCTURE

In [ ]:
cltv_df = pd.DataFrame()

cltv_df["recency_cltv_weekly"] = df.groupby("Customer ID").agg({"InvoiceDate" : lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days}) / 7
cltv_df["T_weekly"] = df.groupby("Customer ID").agg({"InvoiceDate" : lambda InvoiceDate: (today_date - InvoiceDate.min()).days}) / 7
cltv_df["frequency"] = df.groupby("Customer ID").agg({"Invoice" : lambda Invoice: Invoice.nunique()})
cltv_df["monetary"] = df.groupby("Customer ID").agg({"TotalPrice": lambda TotalPrice: TotalPrice.sum()})
cltv_df["monetary_cltv_avg"] = cltv_df["monetary"] / cltv_df["frequency"]
cltv_df.reset_index()

cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

cltv_df.head()

In [ ]:
cltv_df.info()

### BG-NBD Modelinin Kurulması

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

In [ ]:
cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                           frequency = cltv_df["frequency"],
                                           recency = cltv_df["recency_cltv_weekly"],
                                           T = cltv_df["T_weekly"])

cltv_df.head()

### GAMMA-GAMMA Modelinin Kurulması

In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(cltv_df["frequency"],
        cltv_df["monetary_cltv_avg"])

In [ ]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(frequency = cltv_df["frequency"],
                                                                       monetary_value = cltv_df["monetary_cltv_avg"])

cltv_df.head()

### BG-NBD ve GG modeli ile CLTV'nin hesaplanması.

In [ ]:
cltv_df["cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)

cltv_df = cltv_df.sort_values("cltv", ascending=False).reset_index()
cltv_df.head(20)

In [ ]:
cltv_df = pd.merge(cltv_df, df[["Customer ID", "Country"]], on="Customer ID", how="inner").drop_duplicates()

cltv_df = cltv_df[cltv_df["Country"].isin(["United Kingdom"])]

cltv_df["Segment"] = pd.qcut(cltv_df["cltv"], 4, ["D", "C", "B", "A"])